Data Set Convert to CSV File and Convert to LSTM Model:

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
import pandas as pd

file_path = p'/content/drive/MyDrive/TwentyNewsgrous.csv'
df = pd.read_csv(file_path)

print(df.head())

                                                Text  D_Number        Label  \
0  From: mathew <mathew@mantis.co.uk>\nSubject: A...     49960  alt.atheism   
1  From: mathew <mathew@mantis.co.uk>\nSubject: A...     51060  alt.atheism   
2  From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...     51119  alt.atheism   
3  From: mathew <mathew@mantis.co.uk>\nSubject: R...     51120  alt.atheism   
4  From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...     51121  alt.atheism   

                                         Email  \
0                         mantis netcom mantis   
1                         mantis mantis mantis   
2                 dbstu1 edu mimsy umd edu umd   
3                        mantis kepler edu unh   
4  Ibm Watson harder ccr harder ccr watson ibm   

                                             Subject  \
0                      alt atheism atheist resources   
1                alt atheism introduction to atheism   
2                                      gospel dating   


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

texts = df['Text']     # Column that has news text
labels = df['Label']   # Column that has category name

# Fill any potential missing values in the 'Text' column with empty strings
texts = texts.fillna('')

encoder = LabelEncoder()
y = encoder.fit_transform(labels)

tokenizer = Tokenizer(num_words=7000) # Set num_words to None to use all words
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

vocab_size = len(tokenizer.word_index) + 1 # Add 1 for padding

#train test split data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2)
rnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64), # Use vocab_size and remove input_length
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(len(set(y)), activation='softmax')
])
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = rnn_model.evaluate(X_test, y_test)
print(f"RNN Model - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/5
471/471 ━━━━━━━━━━━━━━━━━━━━ 51s 105ms/step - accuracy: 0.0678 - loss: 2.9431 - val_accuracy: 0.1073 - val_loss: 2.6501
Epoch 2/5
471/471 ━━━━━━━━━━━━━━━━━━━━ 49s 103ms/step - accuracy: 0.0988 - loss: 2.7439 - val_accuracy: 0.0797 - val_loss: 2.8986
Epoch 3/5
471/471 ━━━━━━━━━━━━━━━━━━━━ 49s 104ms/step - accuracy: 0.0950 - loss: 2.8561 - val_accuracy: 0.0836 - val_loss: 2.9134
Epoch 4/5
471/471 ━━━━━━━━━━━━━━━━━━━━ 49s 105ms/step - accuracy: 0.0902 - loss: 2.8558 - val_accuracy: 0.0882 - val_loss: 2.8789
Epoch 5/5
471/471 ━━━━━━━━━━━━━━━━━━━━ 49s 105ms/step - accuracy: 0.1019 - loss: 2.7900 - val_accuracy: 0.1176 - val_loss: 2.6335
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.1183 - loss: 2.6317
RNN Model - Loss: 2.633514881134033, Accuracy: 0.1176314428448677


In [14]:
# Predict any custom sentence
sample = ["AI is transforming the world"]
sample_seq = tokenizer.texts_to_sequences(sample)
sample_pad = pad_sequences(sample_seq, maxlen=200)
pred = rnn_model.predict(sample_pad)
label = encoder.inverse_transform([pred.argmax()])
print("Predicted Category:", label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Predicted Category: rec.sport.hockey
